In [2]:
import keras
import seq2seq
from seq2seq.models import AttentionSeq2Seq
import tensorflow as tf
import numpy as np
import pandas as pd

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Get only api call w/o parameters
* deal with X

In [1]:
%run MotifAnalysis/Hooklog3.ipynb
Hooklog = Hooklog3


In [3]:
# input 
in_directory = "./MotifAnalysis/hooklogs/allaple_woj_g_98/" # make sure the last character is '/'
#in_tag = "morstar"
in_tag = in_directory.split("/")[-2]
in_parseFirstPar = False
in_window = 1

# output
out_hl_list = None
out_tag = in_tag

# outfile
out_csvfile = './MotifAnalysis/output/hl_list_'+ out_tag + '.csv'
out_featuretabfile = './MotifAnalysis/output/featurecsvfile_'+ out_tag + '.tab'
out_picklefile = './MotifAnalysis/pickle/hl_list_'+ out_tag + '.pickle'
out_apifreq_pk = './MotifAnalysis/pickle/apifreq_dict_'+ out_tag + '.pickle'

In [4]:
# Example
import os

hl_list = next(os.walk(in_directory))[2] # get all filenames in the in_directory
hl_list = [os.path.join(in_directory, f) for f in hl_list] # filepathname list

hl_list = list(filter(lambda f: f.endswith(".hooklog"), hl_list)) # in case some non-hooklog file in the folder

for file in hl_list:
    hl3 = Hooklog(file, in_parseFirstPar)

In [5]:
# Test

# print last hl3
print("=====")
print(hl3)

# get the api call sequence of last hk3
#print("=====")
#for time, api in hl3.li:
#    print(time, api)

# print each item
for i in hl3.li:
    print(i)

=====
class Hooklog3, ./MotifAnalysis/hooklogs/allaple_woj_g_98/329d35de1e16f1060805b04bc4ee3a3a14c5474d754cd13ad82692a202f53cb9_3328.trace_gSDXp90.hooklog, par = 0, len = 108, digit name = 329d35-gSDXp90
('225320000', 'LoadLibrary')
('225560000', 'LoadLibrary')
('226150000', 'LoadLibrary')
('226430000', 'LoadLibrary')
('226580000', 'LoadLibrary')
('226730000', 'LoadLibrary')
('227030000', 'LoadLibrary')
('227160000', 'LoadLibrary')
('227450000', 'RegQueryValue')
('227450000', 'RegQueryValue')
('227500000', 'LoadLibrary')
('227890000', 'RegQueryValue')
('228160000', 'LoadLibrary')
('228820000', 'CreateFile')
('229050000', 'RegQueryValue')
('229060000', 'RegQueryValue')
('229070000', 'RegEnumValue')
('229340000', 'LoadLibrary')
('229890000', 'RegQueryValue')
('229900000', 'LoadLibrary')
('230230000', 'LoadLibrary')
('230330000', 'LoadLibrary')
('230650000', 'LoadLibrary')
('230840000', 'LoadLibrary')
('231230000', 'CreateFile')
('231240000', 'CreateFile')
('232320000', 'RegCreateKey')
(

In [6]:
# output
import pickle

out_hl_list = hl_list
out_tag = in_tag
with open(out_picklefile, 'wb') as o:
    pickle.dump(out_hl_list, o)

In [13]:
hl3

In [7]:
# output 2
with open(out_csvfile, 'wb') as o:
    o.write(b"hooklog\n")
    for h in hl_list:
        o.write(h.encode("ascii") + b"\n")

In [8]:
# output 3
total_api_set = set()
for file in hl_list:
    hl3 = Hooklog(file, in_parseFirstPar)
    total_api_set.update(hl3.getWinSet(in_window))

with open(out_featuretabfile, 'wb') as o:
    o.write(b"hooklog")
    for api in total_api_set:
        o.write(b"\t" + api.encode("ascii"))
    o.write(b"\n")
    
    for h in hl_list:
        hl3 = Hooklog(h, in_parseFirstPar)
        hl3_set = hl3.getWinSet(in_window)
        o.write(hl3.digitname.encode("ascii"))
        for api in total_api_set:
            if api in hl3_set:
                o.write(b"\tTrue")
            else:
                o.write(b"\tFalse")
        o.write(b"\n")

In [9]:
# output 4
# count api freq
freq_dict = dict()
for file in hl_list:
    hl3 = Hooklog(file, in_parseFirstPar)
    for i in hl3.li:
        if i[1] not in freq_dict:
            freq_dict[i[1]] = 1
        else:
            freq_dict[i[1]] += 1

print(freq_dict)

import pickle
with open(out_apifreq_pk, 'wb') as o:
    pickle.dump(freq_dict, o)

{'LoadLibrary': 550, 'RegQueryValue': 2456, 'CreateFile': 296, 'RegEnumValue': 34, 'RegCreateKey': 64, 'RegSetValue': 64, 'CreateThread': 135, 'CopyFile': 5, 'CreateProcessInternal': 30, 'CreateProcess': 15, 'ExitProcess': 6}


### Build  Stage1Phase1 seq2seq model

In [2]:
model = AttentionSeq2Seq(input_dim=5, input_length=7, hidden_dim=10, 
                         output_length=8, output_dim=20, depth=4)
model.compile(loss='mse', optimizer='rmsprop')

In [ ]:

model.fit()